# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 8:39AM,264624,19,8005000,"AdvaGen Pharma, Ltd",Released
1,Jun 13 2023 8:39AM,264623,20,WT 9566950(8004411) Out,"Exact-Rx, Inc.",Released
2,Jun 13 2023 8:39AM,264623,20,WT 9566957(8004418) Out,"Exact-Rx, Inc.",Released
3,Jun 13 2023 8:39AM,264623,20,9566939,"Exact-Rx, Inc.",Released
4,Jun 13 2023 8:39AM,264623,20,9566942,"Exact-Rx, Inc.",Released
5,Jun 13 2023 8:39AM,264623,20,9566943,"Exact-Rx, Inc.",Released
6,Jun 13 2023 8:39AM,264623,20,9566944,"Exact-Rx, Inc.",Released
7,Jun 13 2023 8:39AM,264623,20,9567499,"Exact-Rx, Inc.",Released
8,Jun 13 2023 8:39AM,264623,20,9567500,"Exact-Rx, Inc.",Released
9,Jun 13 2023 8:39AM,264623,20,9566959,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
4,264620,Released,18
5,264621,Released,2
6,264622,Released,1
7,264623,Released,26
8,264624,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264620,NaN,18.0
264621,NaN,2.0
264622,NaN,1.0
264623,NaN,26.0
264624,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264555,0.0,1.0
264588,0.0,1.0
264604,1.0,0.0
264618,0.0,1.0
264620,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264555,0,1
264588,0,1
264604,1,0
264618,0,1
264620,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264555,0,1
1,264588,0,1
2,264604,1,0
3,264618,0,1
4,264620,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264555,,1
1,264588,,1
2,264604,1,
3,264618,,1
4,264620,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 8:39AM,264624,19,"AdvaGen Pharma, Ltd"
1,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc."
27,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC
28,Jun 13 2023 7:50AM,264621,19,"Methapharm, Inc."
30,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd"
48,Jun 12 2023 4:16PM,264618,19,ACG North America LLC
49,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.
50,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd"
51,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 13 2023 8:39AM,264624,19,"AdvaGen Pharma, Ltd",,1
1,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",,26
2,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
3,Jun 13 2023 7:50AM,264621,19,"Methapharm, Inc.",,2
4,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",,18
5,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,,1
6,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,1,
7,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",,1
8,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 8:39AM,264624,19,"AdvaGen Pharma, Ltd",1,
1,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",26,
2,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
3,Jun 13 2023 7:50AM,264621,19,"Methapharm, Inc.",2,
4,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",18,
5,Jun 12 2023 4:16PM,264618,19,ACG North America LLC,1,
6,Jun 12 2023 2:38PM,264604,16,Sartorius Stedim Filters Inc.,,1
7,Jun 12 2023 1:25PM,264588,19,"AdvaGen Pharma, Ltd",1,
8,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 8:39AM,264624,19,"AdvaGen Pharma, Ltd",1,
1,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",26,
2,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
3,Jun 13 2023 7:50AM,264621,19,"Methapharm, Inc.",2,
4,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 8:39AM,264624,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",26.0,NaN
2,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN
3,Jun 13 2023 7:50AM,264621,19,"Methapharm, Inc.",2.0,NaN
4,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",18.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 8:39AM,264624,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",26.0,0.0
2,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0
3,Jun 13 2023 7:50AM,264621,19,"Methapharm, Inc.",2.0,0.0
4,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",18.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
16,264604,0.0,1.0
19,1587626,24.0,0.0
20,264623,26.0,0.0
22,264622,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,16,264604,0.0,1.0
1,19,1587626,24.0,0.0
2,20,264623,26.0,0.0
3,22,264622,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,16,0.0,1.0
1,19,24.0,0.0
2,20,26.0,0.0
3,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,16,Released,0.0
1,19,Released,24.0
2,20,Released,26.0
3,22,Released,1.0
4,16,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,16,19,20,22
Status,,,,
Executing,1.0,0.0,0.0,0.0
Released,0.0,24.0,26.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,16,19,20,22
0,Executing,1.0,0.0,0.0,0.0
1,Released,0.0,24.0,26.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,16,19,20,22
0,Executing,1.0,0.0,0.0,0.0
1,Released,0.0,24.0,26.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()